# Introducción

Los modelos ARIMA (en alguna de sus variantes) son el enfoque se ha utilizado tradicionalmente para el análsis de series temporales. Iremos uno a uno repasando los modelos que lo componen, junto con un ejemplo final. Estos modelos se pueden usar tanto para hacer pronósticos como para comprender mejor los datos. No cubriremos toda la teoría detrás del modelo ARIMA, pero mostraremos los pasos que debe seguir para aplicarlo correctamente. Estos modelos se pueden utilizar como punto de referencia para la comparación con otros.


### Auto Regresion (AR)

Indica que la serie temporal se regresa sobre sus propios valores retrasados (_retardos_)


### Integrada (I)

Indica que los valores de los datos han sido reemplazados por la diferencia entre los valores y sus retardos para hacer estacionaria la serie.



### Media móvil  (_Moving Average - MA_)
Indica que el error de regresión es en realidad una combinación lineal de términos de error cuyos valores ocurrieron simultáneamente (ej - en el mismo período) y en varios momentos en el pasado.

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
import matplotlib.pyplot as plt
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from matplotlib.pylab import rcParams
from pmdarima import auto_arima
from statsmodels.tsa.arima.model import ARIMA, ARIMAResults
from statsmodels.tsa.statespace.sarimax import SARIMAX
import random


rcParams['figure.figsize'] = 15, 8
plt.style.use('seaborn-v0_8')

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)

# Modelos AR  y MA

Los modelos ARMA tienen dos componentes: Autorreregresivo (AR) y de Media Móvil (MA)

## Autoregressive Models (AR)

El componente autorregresivo se expresa como el valor de un serie (Yt) en términos de sus valores retardos (Yt-1, Yt-2, etc....). Estos valores pasados ​​son representaciones del valor desplazado a períodos anterioes (es decir, _retrasado p periodos_). El término de error -_ruido_- se representa con e. Los valores 𝜙 son los coeficientes del modelo y representan la "influencia" de cada valor pasado en el valor (actual) de la serie temporal.

El proceso AR(p) se representa como sigue:

𝑦𝑡 = 𝜙1𝑦𝑡−1 + 𝜙2𝑦𝑡−2 + … + 𝜙p𝑦𝑡−p + 𝜖𝑡

El orden del modelo viene expresado por p y representa el número de retardos aplicados - el modelo se denomina AR(p).

In [ ]:
# Simulate a first order AR time series
# AR(1) model is just an ARMA(1,0) model

# Import data generation function
from statsmodels.tsa.arima_process import ArmaProcess

# Be sure to flip signs for AR for the arma_generate_sample()
# See https://www.statsmodels.org/dev/generated/statsmodels.tsa.arima_process.ArmaProcess.html for details


# Here we are going to simulate an AR series of order 1, with coefficient 0.8
ar_coeff = np.array([1, -0.8]) # Add zero lag coeff and negate
ma_coeff = np.array([1]) # Add zero lag coeff


# Generate data
ar_series_1 = ArmaProcess(ar_coeff, ma_coeff)
ts_ar1 = ar_series_1.generate_sample(nsample=1000)


px.line(ts_ar1)



### Análisis de estacionariedad

In [ ]:
# Check stationarity

from statsmodels.tsa.stattools import adfuller

result = adfuller(ts_ar1, autolag='AIC') # chooses the number of lags that renders the lowest AIC

#to help you, we added the names of every value
dict(zip(['adf', 'pvalue', 'n_lags', 'n_observations', 'critical' 'values', 'icbest'], result))

In [ ]:
#Another way of checking stationarity of a generated ts
ar_series_1.isstationary

#### Tendencia estructural 

Puede haber tendencias monotónicas que el test de Dicker Fuller puede no capturar. Una tendencia monotónica ascendente (descendente) significa que la variable aumenta (disminuye) de manera consistente a lo largo del tiempo, pero la tendencia puede ser lineal o no y por tanto pueden pasar inadvertidas para tests paramétricos como el de Dickey Fuller. 


In [6]:
import pymannkendall as mk
from statsmodels.stats.diagnostic import het_white

def check_trend(time_series, alpha=0.05):
    """
    Perform the Mann-Kendall test for stationarity on a time series.
    
    H0: No monotonic trend --> Stationary
    Ha : Monotonic trend is present --> Non-Stationary (i.e.  trend exists)
    
    Assumptions
    
    - No correlation
    - No seasonality



    Parameters:
    - time_series: pandas Series or DataFrame column containing the time series data.
    - alpha: Significance level for the test.

    Returns:
    - result: pymannkendall test result object.
    """
    result = mk.original_test(time_series, alpha=alpha)
    return result

In [ ]:
check_trend(ts_ar1, alpha=0.05)

### Autocorrelación y autocorrelación parcial

In [ ]:

# Plot the ACF and PACF of the generated TS
plot_acf(ts_ar1, lags=20)
plot_pacf(ts_ar1, lags=20)
plt.show

### Ajuste del modelo

In [ ]:

# Fit an AR(1) model to the first simulated data
model = ARIMA(ts_ar1, order=(1, 0, 0))
result_model = model.fit()
print(result_model.summary())

### Diagnóstico

In [ ]:

# Get the residuals

px.line(result_model.resid)


In [ ]:
# Absolute error

mae = np.mean(np.abs(result_model.resid))

# Print mean absolute error
print(mae)

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 15, 8


result_model.plot_diagnostics()
plt.show()


¿Cómo interpretar el resultado de `plot_diagnostics()`?

+ **Arriba a la izquierda:** Los errores residuales parecen fluctuar alrededor de una media de cero y tienen una varianza uniforme.
+ **Arriba a la derecha:** El diagrama de densidad sugiere una distribución normal con media cero.
+ **Abajo a la izquierda:** Todos los puntos deben estar perfectamente alineados con la línea roja. Cualquier desviación significativa implicaría que la distribución está sesgada.
+ **Abajo a la derecha:** El correlograma, también conocido como gráfico ACF, muestra que los errores residuales no están autocorrelacionados. Cualquier autocorrelación implicaría que existe algún patrón en los errores residuales que no se explica en el modelo. Una solución consistiría en buscar más X (predictores) para el modelo.

### Predicción

In [ ]:
# Get in-sample predictions --> Use general method: predict()
in_sample_prediction = result_model.predict(return_conf_int=False, start = 0, end = 812, dynamic = False) # could use get_prediction() too 
in_sample_prediction[0:5]

In [ ]:
# Test data
# Get out-sample predictions --> Use ARIMA-specific method: get_forecast()

future_prediction = result_model.get_forecast(steps = 12)

predicted = pd.DataFrame( future_prediction.predicted_mean, columns = ['predicted_future'])


In [ ]:
intervals = pd.DataFrame( future_prediction.conf_int(alpha = 0.1), columns = ['p05', 'p95'])
predicted


In [ ]:
# Concat the information of out sample forecasts 
df_future_results = pd.concat([intervals,predicted], axis = 1)
df_future_results


In [ ]:

# Train data - In sample forecast
# Compare the results with real - observed -  values and the predicted past

df_observed = pd.DataFrame(ts_ar1[0:800], columns = ['real_value'])
df_predicted_past = pd.DataFrame(in_sample_prediction, columns = ['predicted_past'])

df_predicted_past

In [ ]:
# link observed and in-sample forecasts
df_total_past = df_observed.join(df_predicted_past)
df_total_past.head(10)


In [ ]:
# add out-sample information and conf intervals
df_total = pd.concat([df_total_past, df_future_results]).reset_index(drop = True)
df_total.loc[795:812]

In [ ]:
px.line(df_total)

In [ ]:
# Parameters of the model


dict(zip(result_model.param_terms, result_model.params))

## Moving Average Models (MA)

La media móvil (MA) es un proceso en el que el valor actual de una serie temporal, y, se define como una combinación lineal de errores pasados. El término de error - _ruido_ - se representa como e. Tanto el t-1 como el t-2 son retrasos del tiempo en los errores. Los parámetros θ representan la influencia (_peso_) de los errores pasados ​​sobre el valor actual de la serie de tiempo Yt


El proceso MA(q) process se puede describir como:

𝑦𝑡 = 𝜖𝑡 + θ1𝜖𝑡−1 + θ2𝜖𝑡−2 + … + θq𝜖𝑡−q


Por lo general, el orden del modelo con q retrasos requeridos se representat como MA (q).


</b> <font color='orange'> **Ejercicio**:

Simula un modelo MA de orden 1 con coeficiente -0.9 y replica los pasos seguidos con el modelo AR de orden 1 </b> </font>

### Autocorrelación

### Ajuste del modelo

### Diagnóstico

### Predicción

# Modelos ARIMA

Para datos no estacionarios, los valores en el model ARIMA (p, d, q) representan:
+ p: Número de observaciones de retraso incluidas en el modelo, también llamado orden de retraso
+ d: Número de veces que se diferencian las observaciones sin procesar, también llamado grado de diferenciación.
+ q: Tamaño de la ventana del promedio móvil, también llamado orden del promedio móvil.

En caso de tener datos estacionales, también debemos aplicar las diferencias estacionales. En este caso el modelo ARIMA se expresa como SARIMA e incluye un componente estacional (P, D, Q)

## Ejemplo

Utilizamos información referida a los datos de los pasajeros. 

In [20]:
# SSL to overcome problem with ssl whn applying pd.read_csv()
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [12]:
# Use passengers dataset

link_pax = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/airline-passengers.csv'
df_pax =  pd.read_csv(link_pax, parse_dates = ['Month'], index_col = 'Month')
display(df_pax.head(5))
display(df_pax.info())


,Passengers
Month,
1949-01-01,112
1949-02-01,118
1949-03-01,132
1949-04-01,129
1949-05-01,121


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 144 entries, 1949-01-01 to 1960-12-01
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   Passengers  144 non-null    int64
dtypes: int64(1)
memory usage: 2.2 KB


None

In [ ]:
# Define explicitamente la frecuencia del índice 
df_pax.index.freq = 'MS'

In [ ]:
px.line(df_pax['Passengers'])

### Test de estacionariedad

Como se puede observar, hay una tendencia en el tiempo y eso sugiere que los datos no son estacionarios. Sin embargo, para asegurar la estacionariedad de la serie, utilizamos el test Dickey-Fuller.

In [ ]:
result_adf_pass = adfuller(df_pax['Passengers'])
dict(zip(['adf', 'pvalue', 'usedlag', 'nobs', 'critical' 'values', 'icbest'], result_adf_pass))

Para corregir la no estacionariedad, necesitamos diferenciar el valor de nuestra serie de tiempo: restar la observación anterior de la observación actual.

- diferencia(T) = observación(T) — observación(T-1)



In [ ]:
df_pax['diff_pass'] = df_pax['Passengers'] - df_pax['Passengers'].shift(1)
df_pax.head()


In [ ]:
px.line(df_pax['diff_pass'])

La serie parece más estacionaria que antes. Confirmemos con la prueba de Dickey Fuller si ese es efectivamente el caso.

In [ ]:

result_diff_pass = adfuller(df_pax['diff_pass'].dropna())
dict(zip(['adf', 'pvalue', 'usedlag', 'nobs', 'critical' 'values', 'icbest'], result_diff_pass))

Como se puede ver, no podemos rechazar la hipótesis nula porque tenemos un p value> 0.05. Esto sugiere que la serie no es estacionaria y que la tenemos que diferenciar nuevamente (hay que aplicar diferencias hasta obtener una serie temporal estacionaria). Aplicamos las diferencias sobre la primera diferencia aplicada en el paso anterior

In [ ]:
df_pax['diff_pass_2'] = df_pax['diff_pass'] - df_pax['diff_pass'].shift(1)


In [ ]:
px.line(df_pax['diff_pass_2'])

In [ ]:
result_diff_pass_2 = adfuller(df_pax['diff_pass_2'].dropna())
dict(zip(['adf', 'pvalue', 'usedlag', 'nobs', 'critical' 'values', 'icbest'], result_diff_pass_2))

El p value es inferior a 0,05, por lo que podemos rechazar la hipótesis nula. Esto significa que la segunda diferencia es estacionaria y sugiere que una buena estimación del componente diferencial (d) es 2.

In [ ]:
df_pax['seas_diff'] = df_pax['Passengers'] - df_pax['Passengers'].shift(12)


In [ ]:
px.line(df_pax['seas_diff'])

Estos datos muestran cierta estacionalidad, por lo que también debemos estimar la diferencia estacional (D). La diferencia estacional se puede calcular de forma similar a la "diferencia normal" (d) teniendo en cuenta la ventana (período) en la que se produce el patrón estacional (en nuestro ejemplo, 12 meses por año) y restando el valor del período anterior al valor actual. Tenga en cuenta que esta no es la primera diferencia estacional. Si conseguimos que la diferencia estacional sea estacionaria, el valor de la diferencia estacional (D) será 0. De lo contrario, calcularemos la primera diferencia estacional.

+ diferencia estacional (T) = observación (T) - observación (T-12)
+ primera diferencia estacional (T) = diferencia estacional (T) - diferencia estacional (T-1)

In [ ]:
result_seas = adfuller((df_pax['seas_diff']).dropna())
dict(zip(['adf', 'pvalue', 'usedlag', 'nobs', 'critical' 'values', 'icbest'], result_seas))


El p-value es inferior a 0,05, por lo que podemos rechazar la hipótesis nula de no estacionariedad. Eso sugiere usar 0 para el valor D (por ejemplo, la diferencia estacional)

### Test de Autocorrelación

El último paso antes del modelo ARIMA es crear los gráficos de autocorrelación y autocorrelación parcial para ayudarnos a estimar los parámetros p, q, P y Q.

Hay algunas reglas muy útiles para los modelos ARIMA y ARIMA estacional que estamos usando que nos ayudan a estimar los parámetros observando los gráficos de autocorrelación y autocorrelación parcial.

Podemos crear gráficos para la segunda diferencia y la diferencia estacional de nuestra series temporal porque éstas son las series estacionarias que hemos obtenido (d = 2, D = 0).

In [ ]:
df_pax.head(5)

In [ ]:
pax_acf = plot_acf(df_pax['diff_pass_2'].dropna())
pax_pacf = plot_pacf(df_pax['diff_pass_2'].dropna())


No podemos ver cortes claros en ninguno de los gráficos. Sin embargo, hay retrasos significativos en ambos gráficos. Esto sugiere usar un término AR y MA. Experimentemos con p = 1 y q = 1. Necesitamos hacer un análisis similar con el componente estacional.


In [ ]:
#Seasonal componet
pax_seas_acf = plot_acf(df_pax['seas_diff'].dropna())
pax_seas_pacf = plot_pacf(df_pax['seas_diff'].dropna())

### Ajuste del modelo

Tenemos una disminución gradual en el gráfico de autocorrelación y un corte en el gráfico de autocorrelación parcial. Esto sugiere utilizar AR y no exceder el valor de 1 para la parte estacional del ARIMA.

Los valores que hemos elegido pueden no ser óptimos y es posible que deba probar diferentes valores para ajustar el modelo.

In [ ]:
model_pax = SARIMAX(df_pax['Passengers'],order=(1,2,1), seasonal_order=(1, 0, 0, 12))
result_pax = model_pax.fit()
result_pax.summary()

### Diagnóstico

In [ ]:
result_pax.plot_diagnostics()
plt.show()

### Predicción

In [ ]:
display(df_pax.info())

In [ ]:
# Crea dataframe con fechas de test

new_dates_test = pd.date_range('1961-01-01', freq = 'MS', periods=48)


df_pred = pd.DataFrame(pd.to_datetime(new_dates_test))
df_pred.columns = ['Month']
df_pred.set_index('Month', inplace = True)
df_pred['Passengers'] = np.nan
df_pred.rename_axis(None, inplace=True)
df_pred = df_pred.sort_index(ascending=True)
df_pred.index.freq = 'MS'    



In [ ]:

# Create a new dataframe with current data and data from the future
df_now_after = pd.concat([df_pax,df_pred])
display(df_now_after.head(5))
display(df_now_after.tail(5))

In [ ]:
# Add the predictions 

df_now_after['predictions'] = result_pax.predict(start=0,end=191) # total months including out-sample
df_now_after[['Passengers','predictions']].plot()

### Evaluación

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error


# remove the nas and get the RMSE
df_now_after_clean = df_now_after[~(df_now_after['diff_pass_2'].isna()) & ~(df_now_after['predictions'].isna())]
mean_squared_error(df_now_after_clean['Passengers'], df_now_after_clean['predictions'], squared = False) # set false for RMSE

In [ ]:
df_now_after_clean['Passengers'].mean()

# Auto ARIMA

In [ ]:
auto_arima_pax = auto_arima(df_pax['Passengers'],
                       start_P=1,
                       start_q=1,
                       max_p=3,
                       max_q=3,
                       m=12,
                       seasonal=True,
                       D=0,
                       max_d = 3,
                       trace=True,
                       error_action='ignore',
                       suppress_warnings=True,
                       stepwise=True)
auto_arima_pax.summary()

### Ajuste del modelo

In [ ]:
# Fit the best model to the data

model_auto = SARIMAX(df_pax['Passengers'],order=(2, 1, 1),
              seasonal_order=(1, 0, 0, 12),
              enforce_stationarity=False,
              enforce_invertibility=False)
results_auto_arima = model_auto.fit()

In [ ]:

forecast_auto_arima = results_auto_arima.predict(start = len(df_pax),
                           end=len(df_pax)+24,
                           typ='levels').rename('forecast_auto_arima')



In [ ]:
# Plot forecasts 
df_pax['Passengers'].plot(figsize=(12,8),legend=True)
forecast_auto_arima.plot(legend=True)
plt.show()

In [ ]:
diag = results_auto_arima.plot_diagnostics()

En el ejemplo anterior, no hemos dividido el conjunto de datos para entrenar y probar, y los pronósticos se han realizado con datos en los que no tenemos ningún valor observado, por lo que no podemos calcular las métricas de error. Para calcular las métricas de error, se deberá dividir el conjunto de datos en entrenar y probar y calcular el MAE en los pronósticos frente a los valores reales.

## Modelos ARIMAX 

In [ ]:
hospital = pd.DataFrame([1.75, 1.66, 1.65, 1.62, 1.48, 1.77, 1.99, 
                         2.17, 1.57, 1.28, 1.44, 1.06, 1.06, 1.2, 
                         0.89, 1.04, 0.77, 0.54, 1.23, 1.33, 1.65, 
                         1.27, 1.26, 1.4, 1.51, 2.13, 2.35, 2.53, 
                         2.19, 1.72, 1.55, 1.19, 0.96, 1.1, 1.16,
                         1.03, 0.71, 0.82, 1.0, 1.51, 1.25, 1.07,
                         0.69, 1.26, 1.73, 1.76, 1.6, 1.59, 2.32,
                         2.41, 1.95, 1.06, 1.24, 1.61, 1.53, 1.26,
                         0.72, 0.71, 0.59, 0.26, 0.61, 0.66, 0.61, 
                         0.97, 1.2, 1.26, 1.0, 0.58, 1.17, 1.81, 2.13, 
                         1.19, 1.38, 1.54, 1.75, 1.74, 1.39, 0.87, 1.66,
                         1.72, 1.48, 1.73, 1.45, 1.0, 1.23, 1.4, 1.05, 
                         0.67, 0.5, 1.13, 1.74, 2.69, 2.29, 2.28, 2.52, 
                         1.92, 1.91, 1.66, 1.98, 1.9, 1.4, 1.01, 1.21, 1.46, 
                         1.8, 1.3, 1.02, 1.46, 1.6, 1.63, 1.47, 1.37, 1.22,
                         1.38, 1.6, 2.44, 2.45, 2.02, 1.72, 1.49, 1.4,
                         1.32, 1.69, 2.01, 2.24, 1.86, 1.4, 1.67, 2.14, 
                         1.51, 1.09, 1.24, 1.66, 1.28, 0.99, 1.15, 1.28, 
                         0.96, 1.3, 1.28, 1.71, 1.56, 1.17, 1.36, 1.78, 2.08,
                         1.97, 2.0, 1.97, 2.02, 1.59, 1.21, 0.86, 0.19, 0.76, 1.08, 0.8, 0.57, 0.94,
                         1.37, 1.61, 1.96, 1.56, 1.1, 1.6, 1.71, 1.29, 1.55], 
                        columns = ['wait_times_hrs'])
hospital['nurse_count'] = [1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 3.0, 3.0, 7.0, 9.0, 9.0,
                           9.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0,
                           9.0, 9.0, 7.0, 7.0, 5.0, 5.0, 3.0, 3.0, 3.0, 5.0, 
                           5.0, 5.0, 7.0, 7.0, 7.0, 7.0, 7.0, 9.0, 9.0, 9.0, 
                           9.0, 9.0, 9.0, 5.0, 5.0, 5.0, 5.0, 5.0, 1.0, 1.0, 
                           1.0, 3.0, 3.0, 5.0, 5.0, 5.0, 9.0, 11.0, 11.0, 
                           11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 
                           11.0, 9.0, 9.0, 7.0, 7.0, 5.0, 5.0, 5.0, 5.0, 5.0,
                           7.0, 5.0, 5.0, 7.0, 7.0, 7.0, 7.0, 5.0, 7.0, 7.0, 
                           7.0, 7.0, 7.0, 7.0, 3.0, 3.0, 3.0, 3.0, 5.0, 3.0, 3.0,
                           3.0, 3.0, 3.0, 5.0, 5.0, 5.0, 7.0, 7.0, 7.0, 7.0, 7.0,
                           7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 3.0, 3.0, 3.0, 3.0, 3.0, 
                           1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 5.0, 5.0, 
                           5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 3.0, 3.0, 
                           5.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 3.0, 
                           7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 7.0, 
                           5.0, 5.0, 5.0, 3.0]## 

hospital.head()

In [ ]:
px.line(hospital)

In [ ]:
fig =px.scatter(x = hospital['wait_times_hrs'], y = hospital['nurse_count'])
fig.update_traces(marker=dict(size=12,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'))

In [ ]:
model = SARIMAX(hospital['wait_times_hrs'], order=[2, 0, 1], exog = hospital['nurse_count'])

# Fit the model
results = model.fit()

# Print model fit summary
print(results.summary())


In [ ]:
a = results.forecast(steps = 4, exog = [2,3,4,5], alpha= 0.95)
pd.DataFrame(a)

In [ ]:
results.plot_diagnostics()
plt.show()

# Enfoque de Machine Learning

In [63]:
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)


In [ ]:
df_pax

In [ ]:
train = df_pax[:'1959-12-01']
test = df_pax['1960-01-01':]

In [ ]:
train.shape, test.shape

In [ ]:
model_pax = SARIMAX(train['Passengers'],order=(2,1,1), seasonal_order=(1, 0, 0, 12))
result_pax = model_pax.fit()
result_pax.summary()

In [ ]:
result_pax.plot_diagnostics()
plt.show()

In [ ]:
# Forma alternativa de crear un dataframe con fechas futuras

from pandas.tseries.offsets import DateOffset

new_dates = [train.index[-1] + DateOffset(months = x) for x in range(1, len(test))]


df_pred = pd.DataFrame(index = new_dates, columns = train.columns)

display(train.tail())
display(test.head())

In [ ]:
train['dataset'] = 'train'
test['dataset'] = 'test'
df_now_after = pd.concat([train,test])
display(df_now_after.head())
display(df_now_after.tail())



In [ ]:
df_now_after['predictions'] = result_pax.predict(start=0, end=len(train) + len(test))
px.line(df_now_after[['Passengers','predictions']])

In [ ]:
# Compare the results with real - observed -  values and the predicted past

test = df_now_after[df_now_after['dataset'] == 'test'] 

mse = mean_squared_error(test['Passengers'], test['predictions']) # set false for RMSE 
rmse = mean_squared_error(test['Passengers'], test['predictions'], squared = False)
mae = mean_absolute_error(test['Passengers'], test['predictions']) # set false for RMSE 

In [ ]:
metrics_df = pd.DataFrame([mse, rmse, mae], index = ['mse', 'rmse', 'mae'])
metrics_df.columns = ['metrics']
metrics_df

In [ ]:
# add 12 extra new dates to df_now_after with nan values for the predictions starting in 1961-01-01

new_dates = [df_now_after.index[-1] + DateOffset(months = x) for x in range(1, 13)]
df_pred = pd.DataFrame(index = new_dates, columns = df_now_after.columns)
df_pred.head()

df_new_after = pd.concat([df_now_after, df_pred])
df_new_after

In [ ]:
# Out of sample prediction

oos_period = 11 # number of periods to forecast out of sample
df_new_after['predictions'] = result_pax.predict(start=0, end=len(train) + len(test) + oos_period)


In [ ]:
df_new_after['Passengers'] = df_new_after['Passengers'].astype(float)
df_new_after

px.line(df_new_after[['Passengers','predictions']])
